In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision.models import convnext_large, ConvNeXt_Large_Weights
from torchvision import datasets, transforms
from torchvision.io import ImageReadMode
from torch.utils.data import Dataset, DataLoader
# import timm
from PIL import Image
from glob import glob
import csv
import random
import numpy as np
import os
import pandas as pd
from tqdm import tqdm

In [2]:
from transformers import AutoFeatureExtractor, VanForImageClassification

from datasets import load_dataset


In [7]:
import torch
tr = torch.rand(8, 18)

tl = []

for i in range(8):
    tl.append(tr[i])

torch.stack(tl)



tensor([[0.2527, 0.9181, 0.8333, 0.1893, 0.1564, 0.6409, 0.9152, 0.5926, 0.8341,
         0.5269, 0.9052, 0.7313, 0.4788, 0.3649, 0.0941, 0.3578, 0.3976, 0.6400],
        [0.1823, 0.3484, 0.8807, 0.3044, 0.6554, 0.5942, 0.9254, 0.0056, 0.2006,
         0.9971, 0.0894, 0.1721, 0.9957, 0.1415, 0.0236, 0.4866, 0.3507, 0.4073],
        [0.1854, 0.4860, 0.8975, 0.5889, 0.5295, 0.1388, 0.2393, 0.7339, 0.2026,
         0.4787, 0.5412, 0.7629, 0.7527, 0.4862, 0.1667, 0.4564, 0.7644, 0.3471],
        [0.9859, 0.9489, 0.3805, 0.9116, 0.0786, 0.8710, 0.2354, 0.8152, 0.3475,
         0.9796, 0.8056, 0.0580, 0.0285, 0.4905, 0.7371, 0.3947, 0.3131, 0.4421],
        [0.3309, 0.5983, 0.5481, 0.1817, 0.3462, 0.7476, 0.9324, 0.1582, 0.8514,
         0.7884, 0.5777, 0.1882, 0.2621, 0.2357, 0.3076, 0.4109, 0.4666, 0.4793],
        [0.5867, 0.1792, 0.2235, 0.5928, 0.7736, 0.2963, 0.7737, 0.6152, 0.7088,
         0.2112, 0.0759, 0.3146, 0.9719, 0.3602, 0.8070, 0.4891, 0.2408, 0.0433],
        [0.0921, 0.121

In [3]:
device = torch.cuda.current_device()
def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
set_seed(2022)

In [4]:
import gc

gc.collect()
torch.cuda.empty_cache()
print(torch.cuda.memory_summary(device=None, abbreviated=False))
print(torch.cuda.current_device())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |

In [5]:

feature_extractor = AutoFeatureExtractor.from_pretrained("Visual-Attention-Network/van-large")
van_model = VanForImageClassification.from_pretrained("Visual-Attention-Network/van-large")

In [6]:
van_model
van_model(torch.randn(10,3, 256, 256)).logits.shape
model = van_model.eval().to(device)

In [4]:
ConvNext = convnext_large(weights=ConvNeXt_Large_Weights.DEFAULT)

In [5]:
# x = torch.rand(128, 3, 224, 224).to(device)
ConvNeXt = convnext_large(weights=ConvNeXt_Large_Weights.DEFAULT)
ConvNeXt_model = ConvNext.eval()
ConvNeXt_model.to(config.device)
# features = model(x)

ConvNeXt(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 192, kernel_size=(4, 4), stride=(4, 4))
      (1): LayerNorm2d((192,), eps=1e-06, elementwise_affine=True)
    )
    (1): Sequential(
      (0): CNBlock(
        (block): Sequential(
          (0): Conv2d(192, 192, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=192)
          (1): Permute()
          (2): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
          (3): Linear(in_features=192, out_features=768, bias=True)
          (4): GELU(approximate=none)
          (5): Linear(in_features=768, out_features=192, bias=True)
          (6): Permute()
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): CNBlock(
        (block): Sequential(
          (0): Conv2d(192, 192, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=192)
          (1): Permute()
          (2): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
          (3): Linear(in

In [7]:
def feature_extractor(model,x):
    model.to(device)
    f = model.features(x)
    return f.reshape(f.size(0), -1)

In [8]:
# from BagDataset import BagDataset
import sys
bag_dir = "/restricteddata/skincancer_kuk/tiles_20x/Tiles"
processed_slides_dir = "/restricteddata/skincancer_kuk/tiles_20x/Features/Van_20X"
processed_slides_files =sorted(glob(os.path.join(processed_slides_dir, "*","*.csv")))
# print(processed_slides_list)
processed_slides_list = [os.path.basename(slide).split(".")[0] for slide in processed_slides_files]


# print(processed_slides_list)    
bags_list = sorted(glob(os.path.join(bag_dir,"*","*"), recursive=True))
# print(len(bags_list))
# print(bags_list[0:10])
slides_list = bags_list.copy()
# print(slides_list[0:10])
for bag in bags_list:

    basename = os.path.basename(bag)
   
    if basename in processed_slides_list:
        # print(basename)
        slides_list.remove(bag)
# print(slides_list)
# print(len(slides_list))

label_file = '/restricteddata/skincancer_kuk/Scanned_WSI/metadata_workingfile_label.csv'
data = pd.read_csv(label_file)
# print(os.path.basename(bags_list[0]))
print(len(slides_list))
# data.rename(columns = {'slidename':'image_nr', 'filepath':'filename'}, inplace = True)
# data.to_csv(label_file, index=False)
slides_list[0:5]

96


['/restricteddata/skincancer_kuk/tiles_20x/Tiles/1_bas/ICNNBCC00728',
 '/restricteddata/skincancer_kuk/tiles_20x/Tiles/1_bas/ICNNBCC00729',
 '/restricteddata/skincancer_kuk/tiles_20x/Tiles/1_bas/ICNNBCC00730',
 '/restricteddata/skincancer_kuk/tiles_20x/Tiles/1_bas/ICNNBCC00731',
 '/restricteddata/skincancer_kuk/tiles_20x/Tiles/1_bas/ICNNBCC00732']

In [9]:
def get_pos(data_dir):
    import re

    if os.path.exists(data_dir):
        files = sorted(glob(os.path.join(data_dir, '*.png'), recursive=True), key=lambda f: int(re.split("[_ -]",os.path.basename(f))[1]))
    else:
        raise ValueError("{} does not exist".format(dir))

    image_pos_list = []
    for file in files:
        basename = os.path.basename(file).rstrip(".png")
        image_pos = [re.split("[_ -]", basename)[1]] + re.split("[_ -]", basename)[3:7]

        image_pos = [int(x) for x in image_pos]
        image_pos_list.append(image_pos)
    return image_pos_list
    
class BagDataset(Dataset):
    def __init__(self, bag_dir: str, label_file, transforms=None):
        '''
        data_dir:      directory to find files
        label_file:    path to label_file
        transforms:    which transformations should be used for the data set
     
        '''
        super(BagDataset).__init__()
        self.bag_dir=bag_dir
        self.transforms = transforms
        self.files = glob(os.path.join(bag_dir, '*.png'), recursive=True)
        #extract tile position from image name
        self.tile_pos = get_pos(bag_dir)
        #extract bag name from slide name/image folder name
        self.bag_name = os.path.basename(bag_dir)
        #read label file
        metadata = pd.read_csv(label_file)
        #transform word label into integer labels and add to the dataframe
        #extract bag label

        self.bag_label = metadata.loc[metadata["image_nr"] == self.bag_name, 'label'].values[0]
          
    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):

        filepath = self.files[idx]
        tile_pos = self.tile_pos[idx]
    
 
        image = Image.open(filepath)
        image = image.convert('RGB')
                    
        if self.transforms:
  
            image = self.transforms(image)      
        # mean, std = image.mean([0,1]), image.std([0,1])
        # image = transforms.Normalize(mean, std)(image)
        return dict(tile = image, tile_pos = torch.tensor(tile_pos), bag_lable = self.bag_label)

In [10]:
transform = transforms.Compose([transforms.Resize(224),transforms.ToTensor()])
image_dataset = BagDataset(bags_list[0], label_file, transforms=transform)

image_data_loader = DataLoader(
  image_dataset, 
  # batch size is whole datset
  batch_size=128, 
  shuffle=False, 
  num_workers=2)

def mean_std(loader):
  images, lebels,_ = next(iter(loader))
  # shape of images = [b,c,w,h]
  mean, std = images.mean([0,2,3]), images.std([0,2,3])
  return mean, std

def batch_mean_and_sd(loader):
    
    cnt = 0
    fst_moment = torch.empty(3)
    snd_moment = torch.empty(3)

    for images, _,_ in loader:
        b, c, h, w = images.shape
        nb_pixels = b * h * w
        sum_ = torch.sum(images, dim=[0, 2, 3])
        sum_of_square = torch.sum(images ** 2,
                                  dim=[0, 2, 3])
        fst_moment = (cnt * fst_moment + sum_) / (cnt + nb_pixels)
        snd_moment = (cnt * snd_moment + sum_of_square) / (cnt + nb_pixels)
        cnt += nb_pixels

    mean, std = fst_moment, torch.sqrt(snd_moment - fst_moment ** 2)        
    return mean,std
  



In [11]:
def get_simclr_transform(s):
      # get a set of data augmentation transformations as described in the SimCLR paper.
      color_jitter = transforms.ColorJitter(0.8 * s, 0.8 * s, 0.8 * s, 0.2 * s)
      data_transforms = transforms.Compose([transforms.ToPILImage(),
                                             transforms.RandomResizedCrop(size=224),
                                             transforms.RandomHorizontalFlip(),
                                             transforms.RandomApply([color_jitter], p=0.8),
                                             transforms.RandomGrayscale(p=0.2),
                                             transforms.GaussianBlur(kernel_size=int(0.06 * 224)),
                                             transforms.ToTensor()])
      return data_transforms

In [12]:
len(bags_list)

1126

In [13]:
save_path = "/restricteddata/skincancer_kuk/tiles_20x/Features/Van_20X"

# model  = van_model.eval()
# model = model.to(device)


def compute_features(model, bags_list, label_file, save_path,start,stop):

    num_bags = len(bags_list)

    for i in range(start, stop):
        feats_list = []
        bag_dataset = BagDataset(bag_dir=bags_list[i], label_file=label_file, transforms=transform)
        dataloader = DataLoader(bag_dataset, batch_size=128, shuffle=False)
        with torch.no_grad():
            pbar = tqdm(enumerate(dataloader), total=len(dataloader))
            for iteration, batch in pbar:
                tile = batch['tile'].float().to(device)
                # print(f"the shape of batched tiles is {tile.shape}")
                # feats = feature_extractor(model,tile) #for feature extractor
                # feats = model(tile)   # for ConvNext
                feats = model(tile).logits  #for Van
                feats = feats.cpu().numpy()
                
                feats_list.extend(feats)
        
                pbar.set_description('Computed: {}/{} -- {}/{}'.format(i+1, num_bags, iteration+1, len(dataloader)))
        if len(feats_list) == 0:
            print('No valid patch extracted from: ' + bags_list[i])
        else:
            df = pd.DataFrame(feats_list)
            os.makedirs(os.path.join(save_path, bags_list[i].split(os.path.sep)[-2]), exist_ok=True)
            print('Saving to: ' + os.path.join(save_path, bags_list[i].split(os.path.sep)[-2], bags_list[i].split(os.path.sep)[-1] + '.csv'))
            df.to_csv(os.path.join(save_path, bags_list[i].split(os.path.sep)[-2], bags_list[i].split(os.path.sep)[-1]+'.csv'), index=False, float_format='%.4f')

In [14]:
start = 60
stop = 100

compute_features(model, slides_list, label_file, save_path,start,stop)

  0%|          | 0/19 [00:32<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 784.00 MiB (GPU 0; 11.91 GiB total capacity; 1.29 GiB already allocated; 159.00 MiB free; 1.31 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF